In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
batch_s = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dimension = 256  # Latent dimensionality of the encoding space.
number_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/content/ben.txt'

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(number_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 4332
Number of unique input tokens: 71
Number of unique output tokens: 91
Max sequence length for inputs: 96
Max sequence length for outputs: 95


In [6]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [9]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
       decoder_target_data[i, t - 1, target_token_index[char]] = 1.
  decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
  decoder_target_data[i, t:, target_token_index[' ']] = 1.     

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dimension, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [15]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [17]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_s,epochs=epochs, validation_split=0.2)

Epoch 1/100
55/55 [==============================] - 53s 913ms/step - loss: 1.4309 - accuracy: 0.7454 - val_loss: 1.4398 - val_accuracy: 0.6737
Epoch 2/100
55/55 [==============================] - 49s 894ms/step - loss: 0.7672 - accuracy: 0.8209 - val_loss: 1.5889 - val_accuracy: 0.6733
Epoch 3/100
55/55 [==============================] - 49s 898ms/step - loss: 0.6844 - accuracy: 0.8278 - val_loss: 1.2353 - val_accuracy: 0.6792
Epoch 4/100
55/55 [==============================] - 49s 901ms/step - loss: 0.6224 - accuracy: 0.8438 - val_loss: 1.1363 - val_accuracy: 0.7098
Epoch 5/100
55/55 [==============================] - 50s 907ms/step - loss: 0.5640 - accuracy: 0.8569 - val_loss: 1.0671 - val_accuracy: 0.7245
Epoch 6/100
55/55 [==============================] - 50s 907ms/step - loss: 0.5121 - accuracy: 0.8660 - val_loss: 1.0604 - val_accuracy: 0.7353
Epoch 7/100
55/55 [==============================] - 50s 906ms/step - loss: 0.4892 - accuracy: 0.8720 - val_loss: 1.0629 - val_accuracy:

In [18]:
model.save('seq2seq.h5')

In [20]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [21]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    states_value = [h, c]
  return decoded_sentence    

In [23]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: আস্তে যান।

-
Input sentence: Go.
Decoded sentence: আস্তে যান।

-
Input sentence: Go.
Decoded sentence: আস্তে যান।

-
Input sentence: Run!
Decoded sentence: চেঁচানো বন্ধ করো।

-
Input sentence: Run!
Decoded sentence: চেঁচানো বন্ধ করো।

-
Input sentence: Who?
Decoded sentence: কে দৌড়েছিল?

-
Input sentence: Fire!
Decoded sentence: আমাদের ডাকিস।

-
Input sentence: Help!
Decoded sentence: আমাদের ডাকিস।

-
Input sentence: Help!
Decoded sentence: আমাদের ডাকিস।

-
Input sentence: Stop!
Decoded sentence: আমাকে ছেড়ে দিন।

-
Input sentence: Stop!
Decoded sentence: আমাকে ছেড়ে দিন।

-
Input sentence: Stop!
Decoded sentence: আমাকে ছেড়ে দিন।

-
Input sentence: Hello!
Decoded sentence: আমাদের ডাকিস।

-
Input sentence: I see.
Decoded sentence: আমি কিছু টাকা চাই।

-
Input sentence: I try.
Decoded sentence: আমি তোমাকে চিৎকার করতে শুনলাম।

-
Input sentence: Smile.
Decoded sentence: কেউ একজন কাশলো।

-
Input sentence: Smile.
Decoded sentence: কেউ একজন কাশলো।

-
